In [1]:
from utz import *
from njdot import crashes, drivers, occupants, pedestrians, vehicles, END_YEAR, WWW_DOT
from njdot.load import pk_base, INDEX_NAME, normalize, print_hists

from nj_crashes.utils import sql
import sqlite3

In [2]:
%%time
# start_year = END_YEAR - 5
# years = list(range(start_year, END_YEAR))
# kwargs = dict(years=years)

kwargs = dict(y)
kwargs = dict()
c = crashes.load(**kwargs)
min_crash = c.index.min()
d = drivers.load()
d = d[d.crash_id >= min_crash]
o = occupants.load()
o = o[o.crash_id >= min_crash]
p = pedestrians.load()
p = p[p.crash_id >= min_crash]
v = vehicles.load()
v = v[v.crash_id >= min_crash]

Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/crashes.parquet
Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/drivers.parquet
Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/occupants.parquet
Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/pedestrians.parquet
Reading /Users/ryan/c/neighbor-ryan/nj-crashes/njdot/data/vehicles.parquet


CPU times: user 15.4 s, sys: 3.17 s, total: 18.6 s
Wall time: 12.4 s


In [3]:
%%time
all_db = False
crash_idxs = [
    ('severity', 'dt', 'cc', 'mc'),
    ('severity', 'ilat', 'ilon'),
    ('severity', 'icc', 'dt'),
]

for tbl, df in {
    'crashes': c,
#     'drivers': d,
#     'occupants': o,
#     'pedestrians': p,
#     'vehicles': v,
}.items():
    if all_db:
        df.to_sql(tbl, db_url, if_exists='replace')
        with sqlite3.connect(db_path) as con:
            cur = con.cursor()
            if tbl == 'crashes':
                for idx_cols in crash_idxs:
                    sql.add_idx(cur, 'crashes', *idx_cols)
            else:
                sql.add_fk(cur, tbl, 'crash_id', 'crashes', 'id')
            #check(tbl, df)
    else:
        tbl_path = f'{WWW_DOT}/{tbl}.db'
        tbl_url = f'sqlite:///{tbl_path}'
        sql.write(
            df=df, tbl=tbl, db_path=tbl_path,
            idxs=crash_idxs if tbl == 'crashes' else [('crash_id',)],
            rm=True, replace=False,
            page_size=2**16,
        )
        if exists(tbl_path):
            remove(tbl_path)
        df.to_sql(tbl, tbl_url)
        with sqlite3.connect(tbl_path) as con:
            cur = con.cursor()        
            if tbl == 'crashes':
                for idx_cols in crash_idxs:
                    sql.add_idx(cur, 'crashes', *idx_cols)
            else:
                sql.add_idx(cur, tbl, 'crash_id', 'crash_id')

CPU times: user 2min 45s, sys: 1min 22s, total: 4min 8s
Wall time: 7min 20s


In [3]:
c

,dt,year,cc,mc,case,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,alcohol,hazmat,crash_type,tv,road,road_direction,route,Route Suffix,sri,mp,road_system,road_character,road_surface,surface_condition,light_condition,env_condition,road_divided,ttcz,cross_street_distance,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,ramp_route,Ramp To/From Route Direction,speed_limit,speed_limit_cross,olat,olon,cell_phone,Other Property Damage,Reporting Badge No.,horizontal_alignment,road_grade,first_harmful_event,occ,reason,ilon,ilat,geometry,icc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001-12-21 18:34:00,2001,1,1,#2001-17846,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,3,2,CALDERON AVENUE,,<NA>,,,NaN,7,2,2,1,6,1,5,1,100,FE,N,RT 30,,,,25,<NA>,NaN,NaN,False,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,<NA>,<NA>,<NA>,<NA>,No MP,NaN,NaN,b'\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00...,<NA>
1,2001-01-01 09:30:00,2001,1,1,01-00029,1,Absecon,,0,0,0,0,p,B,False,False,6,2,RITZ DRIVE,,<NA>,,,NaN,7,1,2,3,6,1,5,1,<NA>,,,,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>,<NA>,No MP,NaN,NaN,b'\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00...,<NA>
2,2001-04-10 14:44:00,2001,1,1,01-004615,1,Absecon,,0,4,0,0,i,I,False,False,3,2,MORTON AVENUE,,<NA>,,,NaN,7,1,2,1,1,1,5,1,<NA>,AT,,NEW YORK AVENUE,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>,<NA>,No MP,NaN,NaN,b'\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00...,<NA>
3,2001-04-15 13:56:00,2001,1,1,01-004880,1,Absecon,,0,1,0,0,i,B,False,False,1,2,RT 30,,30,,00000030__,51.099998,2,1,2,1,1,1,5,1,<NA>,,,,,,,45,<NA>,NaN,NaN,False,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,<NA>,<NA>,<NA>,<NA>,None,-74.512308,39.432705,b'\x01\x01\x00\x00\x00\xd4\xa9\xef\xa8\xc9\xa0...,1
4,2001-04-16 10:29:00,2001,1,1,01-004912,1,Absecon,,0,0,0,0,p,I,False,False,1,2,CR 651,,651,,,NaN,5,2,2,2,1,2,5,1,<NA>,AT,,CR 630,,,,35,<NA>,NaN,NaN,False,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,<NA>,<NA>,<NA>,<NA>,No MP,NaN,NaN,b'\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00...,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021-05-26 21:08:00,2021,21,23,B150-2021-00117A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.000000,5,<NA>,2,1,6,1,5,1,<NA>,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,False,Damage to 10ft of Country road guardrail.,7837,2,4,46,21,None,-75.075607,40.807156,b'\x01\x01\x00\x00\x00\xa1\x8f\xbe\xbe\xd6\xc4...,21
6077299,2021-05-25 07:26:00,2021,21,23,B150-2021-00118A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,1,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,<NA>,,21231034__,1.730000,7,<NA>,2,1,1,1,5,1,<NA>,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,False,,8610,1,4,26,21,None,-75.056122,40.844345,b'\x01\x01\x00\x00\x00\xdev\x12\x81\x97\xc3R\x...,21
6077300,2021-06-03 20:17:00,2021,21,23,B150-2021-00128A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.000000,5,<NA>,2,2,5,4,5,1,<NA>,,,,,,,50,<NA>,40.80564,-75.07627,False,,8514,3,4,24,21,None,-75.015149,40.863108,b'\x01\x01\x00\x00\x00\x00\xce-2\xf8\xc0R\xc0\...,21


In [11]:
db_path = f'all_{start_year}-{END_YEAR}.db'
db_url = f'sqlite:///{db_path}'

def check(tbl, df1):
    df2 = read_sql(tbl, db_url, index_col='id')
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    assert cols1 == cols2, f"{cols1.difference(cols2)} missing, {cols2.difference(cols1)} extra"
    df2 = df2[df1.columns]
    assert ((df1 == df2) | (df1.isna() & df2.isna())).all().all()
    return df2

In [14]:
c

,dt,year,cc,mc,case,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,alcohol,hazmat,crash_type,tv,road,road_direction,route,Route Suffix,sri,mp,road_system,road_character,road_surface,surface_condition,light_condition,env_condition,road_divided,ttcz,cross_street_distance,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,ramp_route,Ramp To/From Route Direction,speed_limit,speed_limit_cross,olat,olon,cell_phone,Other Property Damage,Reporting Badge No.,horizontal_alignment,road_grade,first_harmful_event,geometry,occ,reason,ilon,ilat,icc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4809915,2017-08-10 17:35:00,2017,1,1,1708-0494,01,Atlantic City Pd,,0,0,0,0,p,B,False,False,1,3,US 30,,30,,00000030__,54.290001,2,<NA>,2,1,1,1,1,1,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,False,,,3,4,26,b'\x01\x01\x00\x00\x00G\xe2\xaf\xc6i\x9fR\xc0p...,<NA>,None,-74.490831,39.393658,1
4809916,2017-08-10 17:36:00,2017,1,1,1708-0497,01,Atlantic City Pd,,0,0,0,0,p,B,False,False,2,2,US 30,W,30,,00000030__,54.290001,2,<NA>,2,1,1,1,1,5,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,False,,530,3,4,26,b'\x01\x01\x00\x00\x00G\xe2\xaf\xc6i\x9fR\xc0p...,<NA>,None,-74.490831,39.393658,1
4809917,2017-09-27 21:27:00,2017,1,1,1709-1131,01,Atlantic City Pd,,0,0,0,0,p,B,False,False,11,1,US 30,W,30,,00000030__,54.290001,2,<NA>,2,1,6,1,1,1,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,False,,530,3,4,48,b'\x01\x01\x00\x00\x00G\xe2\xaf\xc6i\x9fR\xc0p...,<NA>,None,-74.490831,39.393658,1
4809918,2017-09-27 21:28:00,2017,1,1,1709-1132,01,Atlantic City Pd,,0,1,0,0,i,B,False,False,1,2,US 30,W,30,,00000030__,54.290001,2,<NA>,2,1,6,1,1,5,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,False,,530,3,4,26,b'\x01\x01\x00\x00\x00G\xe2\xaf\xc6i\x9fR\xc0p...,<NA>,None,-74.490831,39.393658,1
4809919,2017-12-18 13:21:00,2017,1,1,1712-0555,01,Atlantic City Pd,TRAFFIC,0,0,0,0,p,B,False,False,2,2,US 30,W,30,,00000030__,54.400002,2,<NA>,2,1,1,1,5,1,<NA>,,,,,,,50,<NA>,NaN,NaN,False,,630,1,4,26,b'\x01\x01\x00\x00\x00\xbe\xf6\x04UU\x9fR\xc0\...,<NA>,None,-74.489583,39.392377,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021-05-26 21:08:00,2021,21,23,B150-2021-00117A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.000000,5,<NA>,2,1,6,1,5,1,<NA>,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,False,Damage to 10ft of Country road guardrail.,7837,2,4,46,b'\x01\x01\x00\x00\x00\xa1\x8f\xbe\xbe\xd6\xc4...,21,None,-75.075607,40.807156,21
6077299,2021-05-25 07:26:00,2021,21,23,B150-2021-00118A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,1,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,<NA>,,21231034__,1.730000,7,<NA>,2,1,1,1,5,1,<NA>,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,False,,8610,1,4,26,b'\x01\x01\x00\x00\x00\xdev\x12\x81\x97\xc3R\x...,21,None,-75.056122,40.844345,21
6077300,2021-06-03 20:17:00,2021,21,23,B150-2021-00128A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.000000,5,<NA>,2,2,5,4,5,1,<NA>,,,,,,,50,<NA>,40.80564,-75.07627,False,,8514,3,4,24,b'\x01\x01\x00\x00\x00\x00\xce-2\xf8\xc0R\xc0\...,21,None,-75.015149,40.863108,21


In [9]:
c.dtypes

dt                              datetime64[us]
year                                     int32
cc                                        int8
mc                                        int8
case                                    object
pdc                                     object
pdn                                     object
station                                 object
tk                                        int8
ti                                        int8
pk                                        int8
pi                                        int8
severity                                object
Intersection                            object
alcohol                                   bool
hazmat                                    bool
crash_type                                Int8
tv                                        int8
road                                    object
road_direction                          object
route                                    Int16
Route Suffix 

In [10]:
c.road_character.value_counts(dropna=False).sort_index()

road_character
0          3445
1       3685358
2        532851
3         79089
4        278923
5        187884
6         21956
<NA>    1287797
Name: count, dtype: Int64

In [8]:
c.crash_type.value_counts(dropna=False).sort_index()

crash_type
0           276
1       1671264
2        776875
3        856433
4        100301
5         42280
6        791254
7        166736
8        336399
9         14009
10        33381
11       738090
12       180445
13       102632
14        44236
15        58085
16          619
20        28957
99       131502
<NA>       3529
Name: count, dtype: Int64

In [17]:
def exc(sql):
    with sqlite3.connect(db_path) as con:
        cur = con.cursor()
        print(cur.execute(sql))

In [3]:
con = sqlite3.connect(db_path)
cur = con.cursor()

In [28]:
with sqlite3.connect(db_path) as con:
    cur = con.cursor()
    sql.add_fk(cur, 'o', 'crash_id', 'c', 'id')

ALTER TABLE o ADD COLUMN crash_id2 REFERENCES c(id)
UPDATE o SET crash_id2 = crash_id
ALTER TABLE o DROP COLUMN crash_id
ALTER TABLE o RENAME COLUMN crash_id2 TO crash_id


In [6]:
c

,dt,year,cc,mc,case,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,alcohol,hazmat,crash_type,tv,road,road_direction,route,Route Suffix,sri,mp,road_system,road_character,road_surface,surface_condition,light_condition,env_condition,road_divided,ttcz,cross_street_distance,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,ramp_route,Ramp To/From Route Direction,speed_limit,speed_limit_cross,olat,olon,cell_phone,Other Property Damage,Reporting Badge No.,horizontal_alignment,road_grade,first_harmful_event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001-12-21 18:34:00,2001,1,1,#2001-17846,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,3,2,CALDERON AVENUE,,<NA>,,,NaN,7,2,2,1,6,1,5,1,100,FE,N,RT 30,,,,25,<NA>,NaN,NaN,False,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,<NA>,<NA>,<NA>
1,2001-01-01 09:30:00,2001,1,1,01-00029,1,Absecon,,0,0,0,0,p,B,False,False,6,2,RITZ DRIVE,,<NA>,,,NaN,7,1,2,3,6,1,5,1,<NA>,,,,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>
2,2001-04-10 14:44:00,2001,1,1,01-004615,1,Absecon,,0,4,0,0,i,I,False,False,3,2,MORTON AVENUE,,<NA>,,,NaN,7,1,2,1,1,1,5,1,<NA>,AT,,NEW YORK AVENUE,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>
3,2001-04-15 13:56:00,2001,1,1,01-004880,1,Absecon,,0,1,0,0,i,B,False,False,1,2,RT 30,,30,,00000030__,51.099998,2,1,2,1,1,1,5,1,<NA>,,,,,,,45,<NA>,NaN,NaN,False,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,<NA>,<NA>,<NA>
4,2001-04-16 10:29:00,2001,1,1,01-004912,1,Absecon,,0,0,0,0,p,I,False,False,1,2,CR 651,,651,,,NaN,5,2,2,2,1,2,5,1,<NA>,AT,,CR 630,,,,35,<NA>,NaN,NaN,False,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021-05-26 21:08:00,2021,21,23,B150-2021-00117A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.000000,5,<NA>,2,1,6,1,5,1,<NA>,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,False,Damage to 10ft of Country road guardrail.,7837,2,4,46
6077299,2021-05-25 07:26:00,2021,21,23,B150-2021-00118A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,1,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,<NA>,,21231034__,1.730000,7,<NA>,2,1,1,1,5,1,<NA>,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,False,,8610,1,4,26
6077300,2021-06-03 20:17:00,2021,21,23,B150-2021-00128A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.000000,5,<NA>,2,2,5,4,5,1,<NA>,,,,,,,50,<NA>,40.80564,-75.07627,False,,8514,3,4,24


In [7]:
p

,crash_id,pn,condition,city,state,zip,dob,age,sex,alc_test_given,alc_test_type,alc_test_results,charge1,summons1,traffic_controls,cir1,cir2,dir,act,inj_loc,inj_type,med_refused,safety_used,hospital,status1,cyclist,other,charge2,summons2,charge3,summons3,charge4,summons4,status2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,1,3,ABSECON,NJ,08201,07/28/1990,10,M,,,,,,<NA>,<NA>,<NA>,<NA>,41,1,5,<NA>,<NA>,,1,False,False,,,,,,,<NA>
1,48,31,3,ABSECON,NJ,08201,01/16/1967,34,M,,,,,,<NA>,<NA>,<NA>,<NA>,<NA>,1,4,<NA>,<NA>,,<NA>,True,False,,,,,,,<NA>
2,76,1,2,EGG HARBOR TWP.,NJ,08234,11/16/1952,48,F,N,,,,,<NA>,<NA>,<NA>,<NA>,46,12,3,<NA>,<NA>,,1,False,False,,,,,,,<NA>
3,114,1,4,GALLOWAY TWP.,NJ,08205,03/31/1959,42,F,N,,,SUBPOENA,,<NA>,<NA>,<NA>,<NA>,49,7,<NA>,<NA>,<NA>,,1,False,False,,,,,,,<NA>
4,236,1,4,ABSECON,NJ,08201,04/18/1953,48,M,N,,,,,<NA>,<NA>,<NA>,<NA>,43,1,4,<NA>,<NA>,,1,False,False,,,,,,,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178516,6076961,1,4,WASHINGTON,NJ,07882,,74,F,N,,,,,5,25,<NA>,3,43,6,8,2,1,7102,1,False,False,,,,,,,<NA>
178517,6076969,31,3,WASHINGTON,NJ,07882,,17,M,N,,,,,4,25,<NA>,2,3,11,5,1,8,,1,True,False,,,,,,,<NA>
178518,6076973,1,1,PORT MURRAY,NJ,07865,,59,M,N,,,,,4,73,89,0,36,12,3,<NA>,<NA>,7102,2,False,False,,,,,,,<NA>


In [9]:
p.inj_type.value_counts().sort_index()

inj_type
0     1341
1      382
2     1887
3     3656
4    18298
5    26299
6      134
7     9149
8    73727
Name: count, dtype: Int64

In [10]:
p.inj_loc.value_counts().sort_index()

inj_loc
0      4605
1     19562
2      4744
3       178
4      1947
5      1845
6      8827
7      7902
8     16043
9      2914
10    14169
11    44009
12    11400
Name: count, dtype: Int64

In [11]:
p.condition.value_counts().sort_index()

condition
0      386
1     3686
2     8083
3    46499
4    80893
5      928
Name: count, dtype: Int64

In [12]:
o.condition.value_counts().sort_index()

condition
0     158837
1       9808
2      33894
3     313777
4    1454484
5     762794
Name: count, dtype: Int64

In [87]:
def condition_hist(df, name):
    cond = df.set_index('crash_id').condition.replace('', nan).dropna()
    cond = cond.loc[cond != 1]
    return cond.groupby(lambda x:x).apply(lambda s: ''.join(s.sort_values().astype(str))).rename(name)

In [88]:
%%time
pih = condition_hist(p, 'pih')
pih

CPU times: user 7.39 s, sys: 250 ms, total: 7.64 s
Wall time: 7.56 s


crash_id
7          3
48         3
76         2
114        4
236        4
          ..
6076959    4
6076961    4
6076969    3
6077021    2
6077192    3
Name: pih, Length: 133525, dtype: object

In [89]:
pih.value_counts().sort_index()

pih
0                     354
00                      2
03                      5
033                     1
034                     1
04                     20
044                     1
2                    7666
22                     95
222                    10
2222                    1
22222                   1
2223                    1
2224                    1
223                     4
22333                   1
22334                   1
223344                  1
224                     3
23                     85
233                    12
23333                   1
23334                   1
2334                    2
233444444               1
234                     6
235                     2
24                     40
244                     5
24444                   1
2455                    1
25                      5
3                   44438
33                    587
333                    49
3333                    9
33333                   2
33333333                1
3333334 

In [90]:
%%time
oih = condition_hist(o, 'oih')
oih

CPU times: user 1min 36s, sys: 1.34 s, total: 1min 37s
Wall time: 1min 37s


crash_id
2          3333
3             4
6             4
9            44
13           44
           ... 
6077298       5
6077299      55
6077300       5
6077301       5
6077302    5555
Name: oih, Length: 1670104, dtype: object

In [91]:
oih.value_counts().sort_index()

oih
0                                                                       86095
00                                                                      12815
000                                                                      3793
0000                                                                     1641
00000                                                                    1006
                                                                        ...  
55555555555555555555555555555555555                                         1
55555555555555555555555555555555555555                                      1
55555555555555555555555555555555555555555555                                1
55555555555555555555555555555555555555555555555                             1
55555555555555555555555555555555555555555555555555555555555555555555        1
Name: count, Length: 986, dtype: int64

In [92]:
def deaths_by_crash(df, name):
    cond = df.set_index('crash_id').condition.replace('', nan).dropna()
    cond = cond[cond == 1]
    return cond.groupby(lambda x:x).size().rename(name)

In [93]:
%%time
pkn = deaths_by_crash(p, 'pkn')
pkn

CPU times: user 18.3 ms, sys: 19.5 ms, total: 37.8 ms
Wall time: 38.3 ms


crash_id
1010       1
1102       1
1142       1
3355       1
3486       1
          ..
6069326    1
6070593    1
6074560    1
6075842    2
6076973    1
Name: pkn, Length: 3640, dtype: Int64

In [94]:
%%time
okn = deaths_by_crash(o, 'okn')
okn

CPU times: user 332 ms, sys: 265 ms, total: 597 ms
Wall time: 748 ms


crash_id
77         1
311        1
2779       1
2984       1
2995       2
          ..
6075926    1
6076266    1
6076384    1
6077140    1
6077267    1
Name: okn, Length: 9018, dtype: Int64

In [95]:
c.tk.sum(), okn.sum() + pkn.sum()

(13499, 13494)

In [96]:
%%time
cm = sxs(c, pih, oih, pkn, okn)
cm

CPU times: user 763 ms, sys: 652 ms, total: 1.41 s
Wall time: 2 s


,dt,year,cc,mc,case,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,alcohol,hazmat,crash_type,tv,road,road_direction,route,Route Suffix,sri,mp,road_system,road_character,road_surface,surface_condition,light_condition,env_condition,road_divided,ttcz,cross_street_distance,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,ramp_route,Ramp To/From Route Direction,speed_limit,speed_limit_cross,olat,olon,cell_phone,Other Property Damage,Reporting Badge No.,horizontal_alignment,road_grade,first_harmful_event,pih,oih,pkn,okn
0,2001-12-21 18:34:00,2001,1,1,#2001-17846,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,3,2,CALDERON AVENUE,,<NA>,,,NaN,7,2,2,1,6,1,5,1,100,FE,N,RT 30,,,,25,<NA>,NaN,NaN,False,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
1,2001-01-01 09:30:00,2001,1,1,01-00029,1,Absecon,,0,0,0,0,p,B,False,False,6,2,RITZ DRIVE,,<NA>,,,NaN,7,1,2,3,6,1,5,1,<NA>,,,,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
2,2001-04-10 14:44:00,2001,1,1,01-004615,1,Absecon,,0,4,0,0,i,I,False,False,3,2,MORTON AVENUE,,<NA>,,,NaN,7,1,2,1,1,1,5,1,<NA>,AT,,NEW YORK AVENUE,,,,25,<NA>,NaN,NaN,False,,836,<NA>,<NA>,<NA>,NaN,3333,<NA>,<NA>
3,2001-04-15 13:56:00,2001,1,1,01-004880,1,Absecon,,0,1,0,0,i,B,False,False,1,2,RT 30,,30,,00000030__,51.099998,2,1,2,1,1,1,5,1,<NA>,,,,,,,45,<NA>,NaN,NaN,False,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,<NA>,<NA>,<NA>,NaN,4,<NA>,<NA>
4,2001-04-16 10:29:00,2001,1,1,01-004912,1,Absecon,,0,0,0,0,p,I,False,False,1,2,CR 651,,651,,,NaN,5,2,2,2,1,2,5,1,<NA>,AT,,CR 630,,,,35,<NA>,NaN,NaN,False,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021-05-26 21:08:00,2021,21,23,B150-2021-00117A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.000000,5,<NA>,2,1,6,1,5,1,<NA>,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,False,Damage to 10ft of Country road guardrail.,7837,2,4,46,NaN,5,<NA>,<NA>
6077299,2021-05-25 07:26:00,2021,21,23,B150-2021-00118A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,1,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,<NA>,,21231034__,1.730000,7,<NA>,2,1,1,1,5,1,<NA>,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,False,,8610,1,4,26,NaN,55,<NA>,<NA>
6077300,2021-06-03 20:17:00,2021,21,23,B150-2021-00128A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.000000,5,<NA>,2,2,5,4,5,1,<NA>,,,,,,,50,<NA>,40.80564,-75.07627,False,,8514,3,4,24,NaN,5,<NA>,<NA>
6077301,2021-06-06 00:00:00,2021,21,23,B150-2021-00130A,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,US 46,E,46,,00000046__,7.500000,2,<NA>,2,1,7,1,5,1,<NA>,,,,,,,50,<NA>,40.84149,-75.04968,False,,8365,1,4,24,NaN,5,<NA>,<NA>


In [102]:
pih[pih.str.contains('1')]

Series([], Name: pih, dtype: object)

In [103]:
oih[oih.str.contains('1')]

Series([], Name: oih, dtype: object)

In [120]:
include = '234'
pin = cm.pih.fillna('').apply(lambda s: sum(1 for c in s if c in include))
oin = cm.oih.fillna('').apply(lambda s: sum(1 for c in s if c in include))
ti2 = (pin + oin).rename('ti2')
ti = c.ti
tis = sxs(ti, ti2)
tis

,ti,ti2
0,0,0
1,0,0
2,4,4
3,1,1
4,0,0
...,...,...
6077298,0,0
6077299,0,0
6077300,0,0
6077301,0,0


In [135]:
v

,crash_id,vn,ins_co,owner_state,make,model,color,vy,state,towed,rm_by,imp_loc,dmg_loc,type,body,cir1,cir2,dir,act,ev1,ev2,ev3,ev4,oversize,hit_run,dep,dmg,ev
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,1,426,NJ,NISSAN MAXIMA,,BUR,1991,NJ,D,<NA>,8,7,1,<NA>,25,<NA>,1,3,26,<NA>,<NA>,<NA>,<NA>,False,None,<NA>,<NA>
1,0,2,989,NJ,LINCOLN TOWNCAR,,BK,1996,NJ,D,2,12,12,6,0,4,<NA>,2,3,26,<NA>,<NA>,<NA>,0,False,None,<NA>,<NA>
2,1,1,962,NJ,TOYOTA 4DR,,GRN,1997,NJ,D,1,11,<NA>,1,<NA>,25,<NA>,3,10,28,<NA>,<NA>,<NA>,<NA>,False,None,<NA>,<NA>
3,1,2,<NA>,,,,,0,,,<NA>,0,0,5,0,2,<NA>,1,1,26,<NA>,<NA>,<NA>,0,False,None,<NA>,<NA>
4,2,1,85,NJ,CHEVY CORSICA,,PUR,1996,NJ,T,3,8,15,1,<NA>,25,<NA>,3,1,26,1,<NA>,<NA>,<NA>,False,None,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11450821,6077299,2,962,NJ,HYUNDAI,SONATA,BK,2012,NJ,,2,6,6,1,<NA>,25,<NA>,4,3,26,<NA>,<NA>,<NA>,<NA>,False,1,3,26
11450822,6077300,1,134,NJ,SUBARU,FORRESTER,SL,2015,NJ,,1,11,11,4,<NA>,57,<NA>,3,1,24,<NA>,<NA>,<NA>,2,False,1,3,24
11450823,6077301,1,148,NJ,TOYOTA,COROLLA,BK,2009,NJ,,1,11,11,1,<NA>,57,<NA>,2,1,24,<NA>,<NA>,<NA>,<NA>,False,1,3,24


In [121]:
o

,crash_id,vehicle_id,on,condition,pos,eject,age,sex,inj_loc,inj_type,med_refused,safety_avail,safety_used,airbag,hospital
id,,,,,,,,,,,,,,,
0,0,0,1,<NA>,1,1,38,M,<NA>,<NA>,,4,4,<NA>,
1,0,1,2,<NA>,1,1,63,F,<NA>,<NA>,,4,4,<NA>,
2,1,2,1,<NA>,<NA>,<NA>,<NA>,,<NA>,<NA>,,<NA>,<NA>,<NA>,
3,2,4,1,3,1,1,29,F,6,8,,4,4,<NA>,
4,2,4,2,3,3,1,7,M,8,5,,4,4,<NA>,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13756721,6077301,11450823,1,5,1,1,74,M,<NA>,<NA>,01,11,4,<NA>,
13756722,6077302,11450824,1,5,1,1,23,M,<NA>,<NA>,,11,4,<NA>,
13756723,6077302,11450824,2,5,3,1,23,F,<NA>,<NA>,,11,4,<NA>,


In [134]:
num_drivers = o.loc[o.pos == 1, 'vehicle_id'].value_counts()
num_drivers.value_counts().sort_index()

count
1     9798018
2       25171
3         494
4          66
5          16
6           3
10          1
Name: count, dtype: int64

In [128]:
o.pos.value_counts(dropna=False)

pos
1       9850214
3       1917114
6        616237
4        510321
<NA>     329292
5        167207
2        123462
9         60591
7         56646
8         48354
0         25027
12        23708
10        19094
11         9459
Name: count, dtype: Int64

In [131]:
%%time
ph = o.set_index('vehicle_id').pos.dropna().groupby(lambda x:x).apply(lambda s: ''.join('%x' % p for p in s))
ph

CPU times: user 2min, sys: 1.36 s, total: 2min 1s
Wall time: 2min 3s


vehicle_id
0              1
1              1
4           1346
5              1
6             13
            ... 
11450821       1
11450822       1
11450823       1
11450824     136
11450825       1
Name: pos, Length: 9909407, dtype: object

In [132]:
ph.value_counts().sort_index()

pos
0                                 16456
00                                  163
000                                  53
0000                                 23
00000                                28
                                  ...  
ccccccccccccccccccc                   1
ccccccccccccccccccccc                 1
cccccccccccccccccccccc                1
ccccccccccccccccccccccccccc           1
cccccccccccccccccccccccccccccc        1
Name: count, Length: 5389, dtype: int64

In [117]:
(ti == ti2).value_counts()

True     6076754
False        549
Name: count, dtype: int64

In [119]:
(ti == ti2).value_counts()

True     5953875
False     123428
Name: count, dtype: int64

In [113]:
cm.loc[ti != ti2, ['pih', 'oih']].fillna('').apply(lambda r: ''.join(sorted(r.pih + r.oih)), axis=1).value_counts()

4                      750985
44                     184820
444                     47644
34                      36049
45                      16837
                        ...  
0223344444                  1
0444444444444444444         1
0000000044                  1
033344444                   1
333333445                   1
Name: count, Length: 678, dtype: int64

In [107]:
cm.loc[6077302]

dt                                  2021-06-06 19:33:00
year                                               2021
cc                                                   21
mc                                                   23
case                                   B150-2021-00132A
pdc                                                  02
pdn                             New Jersey State Police
station                                 WASHINGTON - SQ
tk                                                    0
ti                                                    0
pk                                                    0
pi                                                    0
severity                                              p
Intersection                                          B
alcohol                                           False
hazmat                                            False
crash_type                                            3
tv                                              

In [98]:
pk2 = cm.pkn.fillna(0)
ok2 = cm.okn.fillna(0)
tk2 = (pk2 + ok2).rename('tk2')
tk = c.tk
tks = sxs(tk, tk2)
tks

,tk,tk2
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
6077298,0,0
6077299,0,0
6077300,0,0
6077301,0,0


In [100]:
pd.crosstab(ti, ti2).replace(0, '')

ti2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,46,47,49,55,56,64,68
ti,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4285638,172092,143949,50955,20084,9091,3667,1517,687,312,417,59,25,18,19,7,5,3,3,4,3,1,2,1,2,3,1,2,1,3,,1,1,1,,1,,,1,,,,,1,,1,,,,,1
1,55,950389,38919,13992,5499,2482,1031,422,205,102,65,23,10,8,4,4,,1,,,1,,,,,,,,,,,,,,,,,,,1,,,,,1,,,,,,
2,5,63,250244,7534,3001,1280,569,260,121,47,47,11,8,2,1,,2,,1,1,,,,,,,,,,,,,,,,,,,,,,1,,,,,,,,1,
3,2,5,3,70825,1854,784,336,149,70,34,16,11,6,2,3,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1,,,
4,,2,,2,23545,647,247,108,48,33,13,10,,2,1,2,2,1,,,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,,,,,9006,174,79,43,23,14,3,2,2,4,1,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,,,,,,,2131,63,27,12,10,2,4,1,2,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,,,,,,,,908,25,10,12,4,,2,,1,,,,,,,1,,,,,,,1,,,,,,,,,,,,,,,,,,,,,
8,,,,,,,,,418,16,3,2,,2,1,,,,1,1,,,,,,,,,,,,,1,,,,,,,,,,,,,,,,,,


In [99]:
(ti == ti2).value_counts()

True     5593607
False     483696
Name: count, dtype: int64

In [101]:
sxs(cm[['ti', 'pih', 'oih']], pin, oin).loc[(ti == 0) & (ti2 == 1)]

,ti,pih,oih,pih,oih
322,0,NaN,0,0,1
524,0,NaN,0,0,1
3762,0,NaN,0,0,1
3765,0,NaN,0,0,1
3807,0,NaN,0,0,1
...,...,...,...,...,...
6077290,0,NaN,5,0,1
6077292,0,NaN,5,0,1
6077298,0,NaN,5,0,1
6077300,0,NaN,5,0,1
